In [ ]:
import pandas as pd
import numpy as np
import json
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully")

All libraries imported successfully


In [ ]:
# Load training data
df_train_original = pd.read_csv('Employee_training.csv')
print(f"Training data loaded: {len(df_train_original)} records")
print(f"   Mean Assessment_Score: {df_train_original['Assessment_Score'].mean():.2f}")

# Load test data from JSON files
test_data_list = []
test_files = {
    'bench_test_data_unique.json': 50,
    'bench_test_data_500.json': 500,
    'bench_test_data_1000.json': 1000
}

for filepath, expected_count in test_files.items():
    try:
        with open(filepath, 'r') as f:
            test_json = json.load(f)
        test_data_list.extend(test_json)
        print(f"Loaded {filepath}: {len(test_json)} records")
    except:
        print(f"Could not load {filepath}")

df_test_data = pd.DataFrame(test_data_list)
print(f"\nTotal test data collected: {len(df_test_data)} records")
print(f"   Mean Assessment_Score: {df_test_data['Assessment_Score'].mean():.2f}")

Training data loaded: 1299 records
   Mean Assessment_Score: 48.80
Loaded bench_test_data_unique.json: 50 records
Loaded bench_test_data_500.json: 500 records
Loaded bench_test_data_1000.json: 1000 records

Total test data collected: 1550 records
   Mean Assessment_Score: 78.99


In [ ]:
# Combine training and test data
df_combined = pd.concat([df_train_original, df_test_data], ignore_index=True)

# Create binary target: Pass (>=70) or Fail (<70)
def score_to_pass_fail(score):
    return 'Pass' if score >= 70 else 'Fail'

df_combined['Assessment_Result'] = df_combined['Assessment_Score'].apply(score_to_pass_fail)

# Show distribution
result_dist = df_combined['Assessment_Result'].value_counts()
print(f"\nDataset Distribution:")
print(f"   Pass (70-100): {result_dist.get('Pass', 0)} records ({result_dist.get('Pass', 0)/len(df_combined)*100:.1f}%)")
print(f"   Fail (0-69):   {result_dist.get('Fail', 0)} records ({result_dist.get('Fail', 0)/len(df_combined)*100:.1f}%)")
print(f"   Total:         {len(df_combined)} records")


Dataset Distribution:
   Pass (70-100): 1936 records (68.0%)
   Fail (0-69):   913 records (32.0%)
   Total:         2849 records


In [ ]:
# Feature engineering
df_binary_processed = df_combined.copy()

categorical_features_binary = ['Grade', 'Department', 'Primary_Skill', 'Secondary_Skill', 
                               'Course_Category', 'Delivery_Mode', 'Business_Priority', 
                               'Bench_Status', 'Learning_Style', 'Career_Goal', 'Training_Success', 'Training_Course_Name']

numerical_features_binary = ['Skill_Gap_Score', 'Availability_Hours_Per_Week', 'Performance_Rating', 'Duration_Hours', 'Completion_Percentage']

categorical_features_binary = [c for c in categorical_features_binary if c in df_binary_processed.columns]
numerical_features_binary = [c for c in numerical_features_binary if c in df_binary_processed.columns]

# Encode categorical features
label_encoders_binary = {}
for col in categorical_features_binary:
    le = LabelEncoder()
    df_binary_processed[col] = le.fit_transform(df_binary_processed[col].astype(str))
    label_encoders_binary[col] = le

# Encode target
le_target_binary = LabelEncoder()
df_binary_processed['Assessment_Result_Encoded'] = le_target_binary.fit_transform(df_binary_processed['Assessment_Result'])

# Scale numerical features
scaler_binary = StandardScaler()
df_binary_processed[numerical_features_binary] = scaler_binary.fit_transform(df_binary_processed[numerical_features_binary])

# Prepare features and target
X_binary = df_binary_processed[categorical_features_binary + numerical_features_binary]
y_binary = df_binary_processed['Assessment_Result_Encoded']

# Train-test split
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(
    X_binary, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

print(f"\nData Preparation Complete:")
print(f"   Total features: {len(categorical_features_binary)} categorical + {len(numerical_features_binary)} numerical")
print(f"   Training set: {len(X_train_binary)} records")
print(f"   Test set: {len(X_test_binary)} records")


Data Preparation Complete:
   Total features: 12 categorical + 5 numerical
   Training set: 2279 records
   Test set: 570 records


In [ ]:
print("Training Classification Models...\n")

# Train XGBoost
print("Training XGBoost Binary Classifier...")
xgb_binary = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=8, random_state=42, verbosity=0)
xgb_binary.fit(X_train_binary, y_train_binary, verbose=False)
xgb_binary_pred = xgb_binary.predict(X_test_binary)
xgb_binary_acc = accuracy_score(y_test_binary, xgb_binary_pred)
print(f"XGBoost Accuracy: {xgb_binary_acc*100:.2f}%")

# Train Random Forest
print("Training Random Forest Classifier...")
rf_binary = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
rf_binary.fit(X_train_binary, y_train_binary)
rf_binary_pred = rf_binary.predict(X_test_binary)
rf_binary_acc = accuracy_score(y_test_binary, rf_binary_pred)
print(f"Random Forest Accuracy: {rf_binary_acc*100:.2f}%")

# Train Logistic Regression
print("Training Logistic Regression...")
lr_binary = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr_binary.fit(X_train_binary, y_train_binary)
lr_binary_pred = lr_binary.predict(X_test_binary)
lr_binary_acc = accuracy_score(y_test_binary, lr_binary_pred)
print(f"Logistic Regression Accuracy: {lr_binary_acc*100:.2f}%")

# Select best model
best_binary_model = max([('XGBoost', xgb_binary, xgb_binary_pred), 
                         ('Random Forest', rf_binary, rf_binary_pred), 
                         ('Logistic Regression', lr_binary, lr_binary_pred)], 
                        key=lambda x: accuracy_score(y_test_binary, x[2]))

best_binary_name, best_binary_obj, best_binary_pred = best_binary_model

print(f"\n Best Model: {best_binary_name} ({accuracy_score(y_test_binary, best_binary_pred)*100:.2f}%)")

Training Classification Models...

Training XGBoost Binary Classifier...
XGBoost Accuracy: 88.07%
Training Random Forest Classifier...
Random Forest Accuracy: 87.02%
Training Logistic Regression...
Logistic Regression Accuracy: 76.14%

 Best Model: XGBoost (88.07%)


In [ ]:
# Detailed classification report
result_names = le_target_binary.classes_
y_test_result_names = [result_names[i] for i in y_test_binary]
best_result_names = [result_names[i] for i in best_binary_pred]

print("\nDETAILED CLASSIFICATION REPORT:")
print("="*70)
print(classification_report(y_test_result_names, best_result_names))

# Confusion matrix
cm = confusion_matrix(y_test_binary, best_binary_pred)
print("\nCONFUSION MATRIX:")
print("-"*50)
print(f"{'':15} {'Predicted Fail':<20} {'Predicted Pass':<20}")
print(f"{'Actual Fail':<15} {cm[0, 0]:<20} {cm[0, 1]:<20}")
print(f"{'Actual Pass':<15} {cm[1, 0]:<20} {cm[1, 1]:<20}")

# ROC-AUC
try:
    auc_score = roc_auc_score(y_test_binary, best_binary_obj.predict_proba(X_test_binary)[:, 1])
    print(f"\nROC-AUC Score: {auc_score:.4f}")
except:
    print("\nROC-AUC Score: N/A")


DETAILED CLASSIFICATION REPORT:
              precision    recall  f1-score   support

        Fail       0.80      0.84      0.82       183
        Pass       0.92      0.90      0.91       387

    accuracy                           0.88       570
   macro avg       0.86      0.87      0.86       570
weighted avg       0.88      0.88      0.88       570


CONFUSION MATRIX:
--------------------------------------------------
                Predicted Fail       Predicted Pass      
Actual Fail     153                  30                  
Actual Pass     38                   349                 

ROC-AUC Score: 0.9143


In [ ]:
def predict_assessment_result(json_file, model, scaler, label_encs, le_tgt, cat_feats, num_feats):
    with open(json_file, 'r') as f:
        test_data = json.load(f)
    
    print(f"\nPREDICTIONS FOR: {json_file}")
    print(f"{'-'*130}")
    print(f"{'Record':<8} {'Actual Score':<15} {'Actual Status':<15} {'Predicted Status':<18} {'Predicted Range':<18} {'Match':<8}")
    print(f"{'-'*130}")
    
    correct = 0
    pass_count = 0
    pass_correct = 0
    fail_count = 0
    fail_correct = 0
    
    for idx, record in enumerate(test_data, 1):
        actual_score = record.get('Assessment_Score', 0)
        actual_result = 'Pass' if actual_score >= 70 else 'Fail'
        
        if actual_result == 'Pass':
            pass_count += 1
        else:
            fail_count += 1
        
        X_single = pd.DataFrame([record])
        
        for col in cat_feats:
            if col in X_single.columns:
                if col in label_encs:
                    try:
                        X_single[col] = label_encs[col].transform(X_single[col].astype(str))
                    except:
                        X_single[col] = 0
                else:
                    X_single[col] = 0
            else:
                X_single[col] = 0
        
        if len(X_single) > 0:
            X_single[num_feats] = scaler.transform(X_single[num_feats])
        
        X_single_feat = X_single[[col for col in cat_feats + num_feats if col in X_single.columns]]
        
        try:
            pred_encoded = model.predict(X_single_feat)[0]
            pred_result = le_tgt.classes_[pred_encoded]
        except:
            pred_result = 'Pass'
        
        pred_range = '70-100' if pred_result == 'Pass' else '0-69'
        
        is_match = "Yes" if pred_result == actual_result else "No"
        if pred_result == actual_result:
            correct += 1
            if actual_result == 'Pass':
                pass_correct += 1
            else:
                fail_correct += 1
        
        print(f"{idx:<8} {actual_score:<15.1f} {pred_range:<18} {actual_result:<15} {pred_result:<18} {is_match:<8}")
    
    accuracy = (correct / len(test_data)) * 100 if test_data else 0
    pass_accuracy = (pass_correct / pass_count * 100) if pass_count > 0 else 0
    fail_accuracy = (fail_correct / fail_count * 100) if fail_count > 0 else 0
    
    print(f"{'-'*130}")
    print(f"Total Records: {len(test_data)} | Overall Accuracy: {accuracy:.2f}% | Correct: {correct}/{len(test_data)}")
    print(f"Pass Accuracy: {pass_accuracy:.2f}% ({pass_correct}/{pass_count}) | Fail Accuracy: {fail_accuracy:.2f}% ({fail_correct}/{fail_count})\n")
    
    return accuracy, pass_accuracy, fail_accuracy

In [ ]:
print(f"Model Type: {best_binary_name} Binary Classifier")
print(f"Threshold: Score >= 70 = Pass, Score < 70 = Fail")
print(f"Test Set Accuracy: {accuracy_score(y_test_binary, best_binary_pred)*100:.2f}%")
print(f"\nTo test a JSON file: predict_assessment_result('your_file.json', best_binary_obj, scaler_binary, label_encoders_binary, le_target_binary, categorical_features_binary, numerical_features_binary)")

Model Type: XGBoost Binary Classifier
Threshold: Score >= 70 = Pass, Score < 70 = Fail
Test Set Accuracy: 88.07%

To test a JSON file: predict_assessment_result('your_file.json', best_binary_obj, scaler_binary, label_encoders_binary, le_target_binary, categorical_features_binary, numerical_features_binary)


In [ ]:
predict_assessment_result('bench_test_data_1000.json', best_binary_obj, scaler_binary, label_encoders_binary, le_target_binary, categorical_features_binary, numerical_features_binary)


PREDICTIONS FOR: bench_test_data_1000.json
----------------------------------------------------------------------------------------------------------------------------------
Record   Actual Score    Actual Status   Predicted Status   Predicted Range    Match   
----------------------------------------------------------------------------------------------------------------------------------
1        83.0            70-100             Pass            Pass               Yes     
2        90.0            70-100             Pass            Pass               Yes     
3        91.0            70-100             Pass            Pass               Yes     
4        82.0            70-100             Pass            Pass               Yes     
5        83.0            70-100             Pass            Pass               Yes     
6        0.0             0-69               Fail            Fail               Yes     
7        80.0            70-100             Pass            Pass              

(99.4, 100.0, 87.2340425531915)